In [ ]:
# Import libraries
import pymc3 as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load data
file_path = '/content/BrentOilPrices.csv'
data = pd.read_csv(file_path)
data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%y')
data.set_index('Date', inplace=True)

In [ ]:
# Set up Bayesian change-point detection
prices = data['Price'].values

In [ ]:
# Model definition
with pm.Model() as model:
    # Define change point
    switchpoint = pm.DiscreteUniform('switchpoint', lower=0, upper=len(prices))

    # Before and after change point
    mean_before = pm.Normal('mean_before', mu=np.mean(prices[:len(prices)//2]), sigma=10)
    mean_after = pm.Normal('mean_after', mu=np.mean(prices[len(prices)//2:]), sigma=10)

    # Define price model
    price = pm.Deterministic('price', pm.math.switch(switchpoint >= np.arange(len(prices)), mean_before, mean_after))
    observed = pm.Normal('observed', mu=price, sigma=5, observed=prices)

    # Sampling
    trace = pm.sample(1000, tune=500, cores=2)

In [ ]:
# Plot results
plt.figure(figsize=(14, 7))
plt.plot(data.index, prices, label='Brent Oil Price', color='blue')
plt.axvline(data.index[trace['switchpoint'].mean()], color='red', linestyle='--', label='Detected Change Point')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.title('Brent Oil Price with Detected Change Point')
plt.legend()
plt.show()